In [ ]:
import PyPDF2
text = ""
pdfFileObj = open('/Users/mguduri/Downloads/vaccine.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#print(pdf.numPages)
for page in range(pdf.numPages):
    pageObj = pdfReader.getPage(page).extractText()
    text += pageObj
    
#text = text.replace('\n', ' ')
#text = text.replace('in˜uenza', 'influenza')
text = text.replace('In˚uenza', 'influenza') 
text = text.replace('in˚uenza', 'influenza')
text = text.replace('in˜uenza', 'influenza')
#print(text)

In [ ]:
from nltk.tokenize import word_tokenize

tok = word_tokenize(text)
#print(tok)

In [ ]:
### Cleaning the text ###

from nltk.corpus import stopwords
#nltk.download()

punctuations = ['(',')',';',':','[',']',',', '...', '.', '&']

words2 = open('/Users/mguduri/Downloads/words.txt', 'r')
words3 = []
words3 = words2.read()

stop_words = stopwords.words('english')

keywords = [word for word in s if not word in stop_words and not word in punctuations and not word in words3]

In [ ]:
from collections import Counter

cnt = Counter()
words = []
count1 = []
for word in keywords:
    cnt[word] += 1
top = cnt.most_common(10)
for (a, b) in top:
    words.append(a)
    count1.append(b)
print(words, count1)

In [ ]:
import texttable as tt
tab = tt.Texttable()

words = []
numbers = []
headings = ['Word','Number']
tab.header(headings)
for a, b in top:
    words.append(a)
    numbers.append(b)
print(words, numbers)
Word = words
Number = numbers

for row in zip(Word, Number):
    tab.add_row(row)

s = tab.draw()
print (s)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

#arr = np.array(top)
string = []
numbers = []
for a, b in top:
    string.append(a)
    numbers.append(b)
    

#plt.xticks(xn, string)
#plt.figure(figsize=(20,10))
#print(string, numbers)
#plt.bar(string, numbers, facecolor='blue', alpha=0.5)
#xn = range(len(string))
bar_width = 0.8
plt.bar(string, numbers, bar_width)
#plt.rcParams['figure.figsize'] = (13, 11)

#plt.xticks(xn, string)

plt.show()

In [ ]:
### Generating wordcloud from a given PDF Doc ###

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(text)

In [ ]:
### Creaing Summary of a given document ###

from nltk.corpus import reuters
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
import nltk
import datetime, re, sys
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    stemmer=PorterStemmer()
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

token_dict = {}
for article in reuters.fileids():
    token_dict[article] = reuters.raw(article)
        
tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english', decode_error='ignore')
print('building term-document matrix... [process started: ' + str(datetime.datetime.now()) + ']')
sys.stdout.flush()

tdm = tfidf.fit_transform(token_dict.values()) # this can take some time (about 60 seconds on my machine)
#print('done! [process finished: ' + str(datetime.datetime.now()) + ']')

In [ ]:
from random import randint

feature_names = tfidf.get_feature_names()
print('TDM contains ' + str(len(feature_names)) + ' terms and ' + str(tdm.shape[0]) + ' documents')

print ('first term: ' + feature_names[0])
print ('last term: ' + feature_names[len(feature_names) - 1])

for i in range(0, 4):
    print ('random term: ' + feature_names[randint(1,len(feature_names) - 2)])

In [ ]:
import math
from __future__ import division
new_list = ""
article_id = randint(0, tdm.shape[0] - 1)
article_text = text
#for b in a.split('.'):
sent_scores = []
for sentence in nltk.sent_tokenize(a):
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / 5))
sent_scores.sort(key=lambda sent: sent[0])
print ('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    
    new_list += summary_sentence[1]+'\n'

print(new_list)
    #print ('\n*** ORIGINAL ***')
    #print (article_text)